<a href="https://colab.research.google.com/github/nAkshat2103/GraphRAG/blob/main/PersonalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple Implementation of Graph Based Retrieval Augmented Generation using Neo4J and Langchain


In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs langchain_groq


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 18.9 MB/s eta 0:00:00
   ━━━━

In [ ]:
from google.colab import userdata
openaiapi= userdata.get("openai")

In [ ]:
NEO4J_URI="neo4j+s://696dddc5.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="Qi4fFF5k8hhm3ZFuNXBdjztH35pdFZ6kje4Ti2VwVfY"

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=openaiapi
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [ ]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo-0125")

In [ ]:
"""from langchain_experimental.graph_transformers import LLMGraphTransformer
llmt=LLMGraphTransformer(llm=llm)"""

'from langchain_experimental.graph_transformers import LLMGraphTransformer\nllmt=LLMGraphTransformer(llm=llm)'

In [ ]:
"""f = open('/content/abc.txt')
content = f.read()
f.close()
"""


"f = open('/content/abc.txt')\ncontent = f.read()\nf.close()\n"

In [ ]:
"""from langchain.schema import Document
doc=Document(page_content=content)"""

'from langchain.schema import Document\ndoc=Document(page_content=content)'

In [ ]:
"""from langchain.text_splitter import TokenTextSplitter
tokensplitter= TokenTextSplitter(chunk_size=512,chunk_overlap=24)
docs= tokensplitter.split_documents([doc])"""

'from langchain.text_splitter import TokenTextSplitter\ntokensplitter= TokenTextSplitter(chunk_size=512,chunk_overlap=24)\ndocs= tokensplitter.split_documents([doc])'

In [ ]:
"""graph_docs=llmt.convert_to_graph_documents(docs)"""

'graph_docs=llmt.convert_to_graph_documents(docs)'

In [ ]:
"""graph.add_graph_documents(
    graph_docs,
    baseEntityLabel=True,
    include_source=True)
"""

'graph.add_graph_documents(\n    graph_docs,\n    baseEntityLabel=True,\n    include_source=True)\n'

In [ ]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"


In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from typing import Tuple, List, Optional
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain_openai import OpenAIEmbeddings
vector_index= Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel,Field

class Entities(BaseModel):
  """Idetifying Information about Entities"""

  names: List[str] = Field(
      ...,
      description ="All the person, organization or business entities that appear in the text"
  )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entites from the text.",
        ),
        (
            "human",
            "Use the given format to extact information from the following"
            "input: {question}",
        ),
    ]
)

In [ ]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "  Peter Thiel is the founder of PayPal and Founder Fund"}).names

['Peter Thiel', 'PayPal', 'Founder Fund']

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
from typing import List

def gen_full_query(input: str, use_fuzzy: bool = False, operator: str = "AND") -> str:
    full_text_query = ""
    cleaned_input = remove_lucene_chars(input).strip()
    if not cleaned_input:
        return full_text_query

    words = cleaned_input.split()

    if len(words) > 1:
        phrase = f'"{cleaned_input}"'
        full_text_query += phrase + " "

    for word in words[:-1]:
        full_text_query += f"{word} " if not use_fuzzy else f"{word}~ {operator} "

    last_word = words[-1]
    full_text_query += f"{last_word}~" if use_fuzzy and len(last_word) > 2 else last_word

    return full_text_query.strip()



In [ ]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('entity', $query, {limit:5})
            YIELD node, score
            WITH node, score
            ORDER BY score DESC  // Ensure the highest scoring results come first
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": gen_full_query(entity, use_fuzzy=False)},
        )
        result += "\n".join([el['output'] for el in response])
    return result



In [ ]:

print(structured_retriever("Akshat Nigam"))

Akshat Nigam - BIRTHPLACE -> Kanpur
Akshat Nigam - CURRENT_LOCATION -> India
Akshat Nigam - COMMUNICATION_PREFERENCE -> Messaging Apps
Akshat Nigam - PREFERRED_MODE_OF_TRANSPORTATION -> Car
Akshat Nigam - FAVORITE_COLOR -> Black
Akshat Nigam - IMPORTANT_PART_OF_LIFE -> Family
Akshat Nigam - CLOSE_TIES -> Patiala
Akshat Nigam - PROFESSIONAL_INTEREST -> Artificial Intelligence
Akshat Nigam - ENJOYABLE_ACTIVITY -> Tv Shows
Akshat Nigam - ENJOYABLE_ACTIVITY -> Movies
Akshat Nigam - MUSIC_PREFERENCE -> Rap
Akshat Nigam - MUSIC_PREFERENCE -> Rnb
Akshat Nigam - MUSIC_PREFERENCE -> Indie Rock
Akshat Nigam - FOOD_PREFERENCE -> Chicken Dishes
Akshat Nigam - FOOD_PREFERENCE -> Paneer
Akshat Nigam - FOOD_PREFERENCE -> Chole Bhature
Akshat Nigam - FAVORITE_DINING_PLACE -> Mantu
Akshat Nigam - FAVORITE_DINING_PLACE -> Rakesh Biryani
Akshat Nigam - DREAM_DESTINATION -> Yosemite National Park
Akshat Nigam - WEATHER_PREFERENCE -> Cold Weather
Akshat Nigam - SELF-DESCRIPTION -> Ambivert
Akshat Nigam - D

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data


In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableLambda,RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema import HumanMessage, AIMessage
_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
template = """Act like you are Akshat Nigam, Elaborate your answer like you are in an interview for a job:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnableParallel
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "where were you born"})

Search query: where were you born


'I was born and raised in Kanpur, India.'